In [1]:
import json
import re
import pandas as pd
import numpy as np
# from utils.post_process import ProcessPred
import os
import csv
from result_replication_src.gen_nat_scl_eval import ProcGenSclNatRes
from result_replication_src.mvp_eval import ProcMvpRes


In [2]:
gen_scl_nat_file = './final_result/acos-comparison/ours24_gen_scl_nat2.json'
gold_annot_res = './final_result/gold_annotations_fin_3.csv'
mapping_path = './final_result/mappings_for_comparison/gen_sch_nat/gen_sch_nat_map.json'

gen_scl_nat_processor =  ProcGenSclNatRes(gen_scl_nat_file, gold_annot_res, mapping_path)
# gen_scl_nat_processor.calc_comp_em_uoce()
# gen_scl_nat_processor.calc_comp_em_acos(mode='aste')

In [3]:
mvp_file = './final_result/acos-comparison/ours24_mvp.json'
mapping_path = './final_result/mappings_for_comparison/gen_sch_nat/gen_sch_nat_map.json'
mvp_processor =  ProcMvpRes(mvp_file, gold_annot_res)
# mvp_processor.calc_comp_em_uoce()
# mvp_processor.calc_comp_em_acos(mode='aste')

In [4]:
print("ASTE RES: (Component-Level Exact Match)")

p, r, f1 = mvp_processor.calc_comp_em_acos(mode='aste')
print(f"MVP --> precision: {p}, recall: {r}, f1 score: {f1}")

p, r, f1 = gen_scl_nat_processor.calc_comp_em_acos(mode='aste')
print(f"GEN_SCL_NAT--> precision: {p}, recall: {r}, f1 score: {f1}")

print("\n")
print("ASTE RES: (Tuple-Level Exact Match)")
p, r, f1 = mvp_processor.calc_tup_em_acos(mode='aste')
print(f"MVP --> precision: {p}, recall: {r}, f1 score: {f1}")

p, r, f1 = gen_scl_nat_processor.calc_tup_em_acos(mode='aste')
print(f"GEN_SCL_NAT--> precision: {p}, recall: {r}, f1 score: {f1}")

print("\n")


print("ACOS RES: (Component-Level Exact Match)")
p, r, f1 = mvp_processor.calc_comp_em_acos(mode='acos')
print(f"MVP --> precision: {p}, recall: {r}, f1 score: {f1}")

p, r, f1 = gen_scl_nat_processor.calc_comp_em_acos(mode='acos')
print(f"GEN_SCL_NAT--> precision: {p}, recall: {r}, f1 score: {f1}")

print("\n")
print("ACOS RES: (Tuple-Level Exact Match)")
p, r, f1 = mvp_processor.calc_tup_em_acos(mode='acos')
print(f"MVP --> precision: {p}, recall: {r}, f1 score: {f1}")

p, r, f1 = gen_scl_nat_processor.calc_tup_em_acos(mode='acos')
print(f"GEN_SCL_NAT--> precision: {p}, recall: {r}, f1 score: {f1}")


ASTE RES: (Component-Level Exact Match)
MVP --> precision: 0.6126126126126128, recall: 0.6766169154228857, f1 score: 0.6430260047281325
GEN_SCL_NAT--> precision: 0.6025641025641025, recall: 0.7014925373134329, f1 score: 0.6482758620689655


ASTE RES: (Tuple-Level Exact Match)
MVP --> precision: 0.3310810810810811, recall: 0.3656716417910448, f1 score: 0.3475177304964539
GEN_SCL_NAT--> precision: 0.32679738562091504, recall: 0.373134328358209, f1 score: 0.34843205574912894


ACOS RES: (Component-Level Exact Match)
MVP --> precision: 0.5283783783783783, recall: 0.5835820895522387, f1 score: 0.5546099290780141
GEN_SCL_NAT--> precision: 0.4961538461538461, recall: 0.5776119402985074, f1 score: 0.5337931034482758


ACOS RES: (Tuple-Level Exact Match)
MVP --> precision: 0.12837837837837837, recall: 0.1417910447761194, f1 score: 0.13475177304964536
GEN_SCL_NAT--> precision: 0.03205128205128205, recall: 0.03731343283582089, f1 score: 0.034482758620689655


In [5]:
print("UOC RESULTS (COMPONENT LEVEL)")

print("\n")

p, r, f1 = mvp_processor.calc_comp_em_uoce()
print(f"MVP --> precision: {p}, recall: {r}, f1 score: {f1}")

print("\n")
p, r, f1 = gen_scl_nat_processor.calc_comp_em_uoce()
print(f"GEN_SCL_NAT--> precision: {p}, recall: {r}, f1 score: {f1}")

UOC RESULTS (COMPONENT LEVEL)


MVP --> precision: 0.35608108108108094, recall: 0.3932835820895521, f1 score: 0.3737588652482268


GEN_SCL_NAT--> precision: 0.39102564102564075, recall: 0.45522388059701463, f1 score: 0.4206896551724135


In [7]:
import sys
sys.path.append('./evaluations/') 

GOLD_PATH = './final_result/gold_annotations_fin_3.csv'
# PRED_PATH = '../ontology_experiments/results/final_results/llama_70B_owl.csv'
from opinion_overlap_eval import OpinionEval
import os
dict_pol = {
    'POS': 'positive',
    'NEG': 'negative',
    'NEU': 'neutral'
}
columns_incls = ['id', 'raw_text', 'aspect_term', 'sentiment_expression', 'target_entity', 'aspect_category', 'sentiment_polarity', 'opinion_holder_span', 'opinion_holder_entity', 'sentiment_intensity', 'opinion_qualifier', 'opinion_reason']

In [9]:
pred_model_dir = './final_result/'
mods = os.listdir(pred_model_dir)
owl_res = 'onto_prompt'
nat_lang_res = 'nlprompt'
op_eval = OpinionEval(columns_incls)
gold_df = pd.read_csv(GOLD_PATH).replace({'sentiment_polarity': dict_pol})
fine_res_rows = []
for mod in mods:
    
    if mod not in ['.DS_Store', 'acos-comparison', 'gold_annotations_fin_3.csv', 'mappings_for_comparison']:
        mod_dir_path = os.path.join(pred_model_dir, mod)
        
        owl_res_path = os.path.join(mod_dir_path, owl_res, 'csv')
        nat_lang_path = os.path.join(mod_dir_path, nat_lang_res, 'csv')
        for owl_file in os.listdir(owl_res_path):
            pred_df = pd.read_csv(os.path.join(owl_res_path, owl_file))
            scores = op_eval.eval_opinions(gold_df, pred_df)
            fine_res_rows.append([mod, 'owl', owl_file.replace('.csv', ''), round(scores['precision']*100, 2), round(scores['recall']*100, 2), round(scores['f1']*100, 2)])
        for nat_file in os.listdir(nat_lang_path):
            pred_df = pd.read_csv(os.path.join(nat_lang_path, nat_file))
            scores = op_eval.eval_opinions(gold_df, pred_df)
            fine_res_rows.append([mod, 'natlang', nat_file.replace('.csv', ''), round(scores['precision']*100, 2), round(scores['recall']*100, 2), round(scores['f1']*100, 2)])




In [10]:
df = pd.DataFrame(fine_res_rows, columns=['model', 'prompt', 'variation',  'precision', 'recall', 'f1'])
owl_res = df[df['prompt'] == 'owl']
natlang_res = df[df['prompt'] == 'natlang']
owl_rearg = owl_res.pivot_table(values='f1', index=['model'], columns='variation')
natlang_rearg = natlang_res.pivot_table(values='f1', index=['model'], columns='variation')

In [11]:
print('Ontoprompt Scores with Different Serialisation Formats')
owl_rearg

Ontoprompt Scores with Different Serialisation Formats


variation,jsonld,manchester_owl,obo_format,owl_funct,owl_xml,rdf_xml,ttl_syntax
model,,,,,,,
gemma2_27b,47.80,47.12,47.99,46.24,48.30,46.13,48.97
gemma2_9b,45.55,45.63,45.10,36.40,45.15,37.07,45.64
gpt-4o,48.09,47.01,47.89,48.04,46.67,47.87,48.44
gpt-4o-mini,46.05,45.31,43.92,44.95,45.26,44.77,44.78
llama_70b,42.83,41.93,43.50,43.05,41.18,42.72,42.42
llama_8b,41.32,42.42,41.16,40.87,41.18,41.98,41.09
mistral_7b,40.89,39.98,39.92,39.54,40.44,39.37,39.39
mixtral_8x7b,43.12,42.32,41.98,41.88,42.19,40.99,42.80


In [12]:
print('NLPROMPT Scores with Different Serialisation Formats')
natlang_rearg

NLPROMPT Scores with Different Serialisation Formats


variation,desc_eg_format,desc_format_eg,eg_desc_format,eg_format_desc,format_desc_eg,format_eg_desc
model,,,,,,
gemma2_27b,48.08,46.60,47.30,47.30,45.96,44.70
gemma2_9b,47.67,46.54,48.80,48.67,46.13,45.39
gpt-4o,48.72,46.32,49.27,49.44,47.96,47.30
gpt-4o-mini,45.56,44.90,46.32,47.50,44.41,46.89
llama_70b,39.08,38.35,40.04,36.78,37.38,38.56
llama_8b,38.63,41.57,36.54,37.27,40.66,29.62
mistral_7b,40.00,40.44,40.90,40.38,41.34,41.92
mixtral_8x7b,41.35,42.14,43.20,42.71,41.34,42.49
